In [27]:
import numpy as np

from uf3.data import composition
from ase import Atoms

# Initializing ChemicalSystem

```element_list``` should contain all elements expected to appear in the experiment. Elements may be provided as either symbols (str) or atomic numbers (int).

```degree``` indicates the highest degree of n-body interactions to consider (i.e. where to truncate the many-body expansion). 

A value of two means the ChemicalSystem will handle pair-wise elemental interactions:
- A-A
- A-B ⟷ B-A
- B-B

A value of three means the ChemicalSystem will handle three-body elemental interactions:
- A-A-A
- A-B-A ⟷ A-A-B
- A-B-B
- B-A-A
- B-A-B ⟷ B-B-A
- B-B-B

Symmetric interactions, denoted with "⟷", are treated as equivalent.

Provided elements and all interactions are sorted by electronegativity.

In [28]:
element_list = ["Na", "Cl"]  # set of elements in system (str or int)
degree = 2

chemical_system = composition.ChemicalSystem(element_list, 
                                             degree=degree)
print(chemical_system)

ChemicalSystem:
    Elements: ('Na', 'Cl')
    Degree: 2
    Pairs: [('Na', 'Na'), ('Na', 'Cl'), ('Cl', 'Cl')]


In [105]:
element_list = ["Cl", "Na"]  # set of elements in system (str or int)
degree = 3

chemical_system = composition.ChemicalSystem(element_list, 
                                             degree=degree)
print(chemical_system)

ChemicalSystem:
    Elements: ('Na', 'Cl')
    Degree: 3
    Pairs: [('Na', 'Na'), ('Na', 'Cl'), ('Cl', 'Cl')]
    Trios: [('Na', 'Na', 'Na'), ('Na', 'Na', 'Cl'), ('Na', 'Cl', 'Cl'), ('Cl', 'Na', 'Na'), ('Cl', 'Na', 'Cl'), ('Cl', 'Cl', 'Cl')]


# ChemicalSystem interactions



In [107]:
chemical_system.interactions_map

{1: ('Na', 'Cl'),
 2: [('Na', 'Na'), ('Na', 'Cl'), ('Cl', 'Cl')],
 3: [('Na', 'Na', 'Na'),
  ('Na', 'Na', 'Cl'),
  ('Na', 'Cl', 'Cl'),
  ('Cl', 'Na', 'Na'),
  ('Cl', 'Na', 'Cl'),
  ('Cl', 'Cl', 'Cl')]}

The one-body interaction can be used to fit the reference elemental energy.
In VASP, this energy depends on the choice of pseudopotential and is equivalent to the energy of an isolated atom.

In practice, chemical and material properties depend on energy differences.
The various derivatives are also unaffected by the value of one-body terms.

Therefore, for molecular dynamics, these energy offsets can be neglected.
Currently, our LAMMPS interface does not use one-body terms.

In [108]:
n_elements = len(chemical_system.element_list)

assert len(chemical_system.interactions_map[1]) == n_elements

The number of pair interactions is equal to $C^R(n, 2)$ (combinations with replacement).

$$C^R(n,r) = \frac{(n + r - 1)!}{ r! (n - 1)! }$$

In [116]:
n_pairs = (np.math.factorial(n_elements + 2 - 1) / 2
           / np.math.factorial(n_elements - 1))
n_pairs = int(n_pairs)
n_pairs = int(numerator / denominator)

assert n_pairs == 3

One may visualize the set of three-body interactions as the cartesian product of the set of elements with the set of pair interactions.

In general, the number of unique n-body interactions is given by the number of unique elements
multiplied by the number of unique (n-1)-body interactions.

In [120]:
n_trios = int(n_pairs * n_elements)
print(n_trios)

6


# Serialization

In [137]:
chemical_system.as_dict()

{'element_list': ('Na', 'Cl'), 'degree': 3}

In [138]:
loaded_system = composition.ChemicalSystem.from_dict({'element_list': ('Na', 'Cl'), 
                                                      'degree': 3})

In [139]:
assert loaded_system.element_list == ("Na", "Cl")
assert loaded_system.degree == 3

# Szudzik Hashes

Internally, n-body element combinations are often represented as integers.

These integer hashes are obtained using the Szudzik pairing function.

\begin{equation}
H(x, y)=
    \begin{cases}
        x^2 + y & \text{if } x > y\\
        y^2 + x + y & \text{if } x \leq y
    \end{cases}
\end{equation}

Trivariate pairing is handled sequentially.

$$H(x, y, z) = H(H(x, y), z)$$

In [129]:
chemical_system.interaction_hashes

{2: array([143, 317, 323]),
 3: array([ 20460,  20466, 100506,  90011,  90017, 104346])}

In [130]:
assert composition.symbols_to_hash(("Na", "Na")) == 143

In [131]:
assert composition.symbols_to_hash(("Na", "Na", "Cl")) == 20466

In [136]:
assert composition.hash_to_symbols(317, n=2) == ("Na", "Cl")

In [134]:
assert composition.hash_to_symbols(90017, n=3) == ("Cl", "Na", "Cl")